# This notebook will be mainly used for the capstone project week 3 - Segmenting and Clustering Neighborhood in Toronto.

## 1 - Creating a dataframe with webscraping.

### 'requests' library is used along with 'BeautifulSoup' for webscraping the necessary data, and 'pandas' is used for converting the data into a dataframe.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


### Requisition to get the url raw data.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html_data = r.text
# html_data


### Changes the raw data into a beautiful soup object, with the 'lxml' parsing type.

In [3]:
soup = BeautifulSoup(html_data, 'lxml')
# print(soup.prettify())


### From the beautiful soup object, finds the 'html' 'table' objects.

In [4]:
table = soup.find('table')
# print(table.prettify())


### First, an empty list named 'table_contents' is created. In this list it will be stored information about the 3 necessary columns: 'PostalCode', 'Borough' and 'Neighborhood' . 

### Then, from the 'table' object created above, and with the aid of beautiful soup, it will be searched all the 'html' 'td' fields. 

### Finally, the list 'table_contents' will be updated with a dictionary called 'cell' for each row that is found within 'td' 'html' field. Observation: Knowing that some readings from beautiful soup may come with unwanted characters, such as '(', ')' and '/', the replace method will be used to handle that.

In [5]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
# table_contents


### With the list of dictionaries 'table_contents' now created and updated, it is possible to load and transform that into a pandas dataframe.

### Some strings will be replaced by a shorter name during this stage.

In [6]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest',
                                     'East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


### Shows the first 5 elements of the dataframe with method '.head()'.

In [7]:
df.head()


PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [8]:
df.shape


(103, 3)

## 2 - Getting longitude and latitude data.

### The below code was very unstable due to the API (geocoder) used.

In [9]:
# import geocoder


In [10]:
# lat_lng_coords = None

# postal_code = 'M4A'

# # loop until the coordinates is given
# while(lat_lng_coords is None):
#   g = geocoder.google(f'{postal_code}, Toronto, Ontario')
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


### So, in order to solve the api problem, a CSV file will be loaded instead of using the API.

In [11]:
df_coordinates = pd.read_csv('datasets/Geospatial_Coordinates.csv')
df_coordinates.head()


Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

### The index of the dataframes will be set as the "Postal Code", because its ID is the same in both dataframes, so it is easier to group by this ID.

In [12]:
df = df.set_index('PostalCode')
df.head()


Borough                      Neighborhood
PostalCode                                                    
M3A               North York                         Parkwoods
M4A               North York                  Victoria Village
M5A         Downtown Toronto         Regent Park, Harbourfront
M6A               North York  Lawrence Manor, Lawrence Heights
M7A             Queen's Park     Ontario Provincial Government

In [13]:
df_coordinates = df_coordinates.set_index('Postal Code')
df_coordinates.head()


Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476

### With the indexes of both dataframes adjusted, it is time to merge them into a single dataframe. 

In [14]:
df_merged = pd.merge(df, df_coordinates, 
                     left_index=True, 
                     right_index=True, 
                     how='outer')
df_merged.head()


Borough                            Neighborhood   Latitude  Longitude
M1B  Scarborough                          Malvern, Rouge  43.806686 -79.194353
M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535 -79.160497
M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573 -79.188711
M1G  Scarborough                                  Woburn  43.770992 -79.216917
M1H  Scarborough                               Cedarbrae  43.773136 -79.239476

### Now that it is already merged, time to return the index: "Postal Code" into the column it was before.

In [15]:
df_merged = df_merged.reset_index().rename(columns={'index':'PostalCode'})
df_merged.head()


PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## 3 - Exploring and clustering the neightborhoods in Toronto.

### Importing the Nominatim module from geopy.

In [16]:
from geopy.geocoders import Nominatim


In [17]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto city are {latitude}, {longitude}.')


The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top. For that, folium is required.

In [18]:
import folium


In [19]:
# create map of Toronto latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto


In [20]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], 
                                           df_merged['Longitude'], 
                                           df_merged['Borough'], 
                                           df_merged['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


### I will be working with only boroughs that contains Toronto as a word, so a filtering will be made.

In [21]:
toronto_data = df_merged[df_merged['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()


PostalCode                 Borough                    Neighborhood  \
0        M4E            East Toronto                     The Beaches   
1        M4J  East York/East Toronto              The Danforth  East   
2        M4K            East Toronto    The Danforth West, Riverdale   
3        M4L            East Toronto  India Bazaar, The Beaches West   
4        M4M            East Toronto                 Studio District   

    Latitude  Longitude  
0  43.676357 -79.293031  
1  43.685347 -79.338106  
2  43.679557 -79.352188  
3  43.668999 -79.315572  
4  43.659526 -79.340923

### Reploting the same map as before, but this time filtered with only the Torontos' words.

In [22]:
filtered_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], 
                                           toronto_data['Longitude'], 
                                           toronto_data['Borough'], 
                                           toronto_data['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(filtered_toronto_map)  

filtered_toronto_map


### For the safety of my own privacy, the credentials used in the below code will be ommited.

In [ ]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '...' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Exploring the first neighborhood in our dataframe.

In [24]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

### In this case, the first neighborhood is called "The Beaches".

In [25]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print(f'Latitude and longitude values of {neighborhood_name} are {neighborhood_latitude}, {neighborhood_longitude}.')


Latitude and longitude values of The Beaches are 43.6763574, -79.2930312.


### Now, let's get the top 100 venues that are in "The Beaches" within a radius of 500 meters.

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # in meters
url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={neighborhood_latitude},{neighborhood_longitude}&radius={radius}&limit={LIMIT}'

# get the result to a json file
results = requests.get(url).json()


In [27]:
results = requests.get(url).json()
# results


In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


### Now we are ready to clean the json and structure it into a pandas dataframe. For that. we import pandas json module handling as well as json module.

In [29]:
import json
from pandas.io.json import json_normalize


In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


<ipython-input-30-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


name         categories        lat        lng
0  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
1                 Grover Pub and Grub                Pub  43.679181 -79.297215
2                       Upper Beaches       Neighborhood  43.680563 -79.292869

In [31]:
print(f'{nearby_venues.shape[0]} venues were returned by Foursquare.')


3 venues were returned by Foursquare.


## Exploring neighborhoods in Toronto. 

### We are using a region that contains Downtown, East, North and Central Toronto in the dataframe. The below function will repeat the same process to all neighborhoods in the listed cardinalities of Toronto.

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url =f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [33]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                 )


The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

### Let's check how many venues were returned for each neighborhood.

In [34]:
toronto_venues.groupby('Neighborhood').count()


Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            46   
Brockton, Parkdale Village, Exhibition Place                           21   
CN Tower, King and Spadina, Railway Lands, Harb...                     18   
Central Bay Street                                                     63   
Christie                                                               15   
Church and Wellesley                                                   66   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             27   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           14   
Enclave of M4L                                                         17   
Enclave of M5E                                                         85   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          20   
India Bazaar, The Beaches West                                         17   
Kensington Market, Chinatown, Grange Park                              57   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               43   
North Toronto West                                                     13   
Parkdale, Roncesvalles                                                 11   
Regent Park, Harbourfront                                              42   
Richmond, Adelaide, King                                               99   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     36   
St. James Town                                                         80   
St. James Town, Cabbagetown                                            38   
Studio District                                                        25   
Summerhill West, Rathnelly, South Hill, Forest ...                      9   
The Annex, North Midtown, Yorkville                                    17   
The Beaches                                                             3   
The Danforth  East                                                      3   
The Danforth West, Riverdale                                           42   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             46   
Brockton, Parkdale Village, Exhibition Place                            21   
CN Tower, King and Spadina, Railway Lands, Harb...                      18   
Central Bay Street                                                      63   
Christie                                                                15   
Church and Wellesley                                                    66   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              27   
Davisville North                                                         8   
Dufferin, Dovercourt Village                                            14   
Enclave of M4L                     

### Let's check the size of the resulting dataframe.

In [35]:
print(toronto_venues.shape)
toronto_venues.head()


(1484, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         The Beaches              43.676357              -79.293031   
1         The Beaches              43.676357              -79.293031   
2         The Beaches              43.676357              -79.293031   
3  The Danforth  East              43.685347              -79.338106   
4  The Danforth  East              43.685347              -79.338106   

                                Venue  Venue Latitude  Venue Longitude  \
0  The Big Carrot Natural Food Market       43.678879       -79.297734   
1                 Grover Pub and Grub       43.679181       -79.297215   
2                       Upper Beaches       43.680563       -79.292869   
3                            The Path       43.683923       -79.335007   
4                  Sammon Convenience       43.686951       -79.335007   

      Venue Category  
0  Health Food Store  
1                Pub  
2       Neighborhood  
3               Park  
4  Convenience Store

### Let's find out how many unique categories can be curated from all the returned venues.

In [36]:
print(f"There are {len(toronto_venues['Venue Category'].unique())} uniques categories.")


There are 220 uniques categories.


### Analysing each neighborhood.

In [37]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


Yoga Studio  Adult Boutique  Afghan Restaurant  Airport  \
0            0               0                  0        0   
1            0               0                  0        0   
2            0               0                  0        0   
3            0               0                  0        0   
4            0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Theme Restaurant  \
0                 0                    0  ...                 0   
1                 0                    0  ...                 0   
2                 0                    0  ...                 0   
3                 0                    0  ...                 0   
4                 0                    0  ...                 0   

   Thrift / Vintage Store  Toy / Game Store  Trail  Train Station  \
0                       0                 0      0              0   
1                       0                 0      0              0   
2                       0                 0      0              0   
3                       0                 0      0              0   
4                       0                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wine Shop  
0         0          0  
1         0          0  
2         0          0  
3         0          0  
4         0          0  

[5 rows x 220 columns]

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()


Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.000000   
2  CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
3                                 Central Bay Street     0.015873   
4                                           Christie     0.000000   

   Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0             0.0                0.0  0.000000            0.000000   
1             0.0                0.0  0.000000            0.000000   
2             0.0                0.0  0.055556            0.055556   
3             0.0                0.0  0.000000            0.000000   
4             0.0                0.0  0.000000            0.000000   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  ...  \
0      0.000000        0.000000         0.000000          0.000000  ...   
1      0.000000        0.000000         0.000000          0.000000  ...   
2      0.055556        0.111111         0.166667          0.111111  ...   
3      0.000000        0.000000         0.000000          0.000000  ...   
4      0.000000        0.000000         0.000000          0.000000  ...   

   Theme Restaurant  Thrift / Vintage Store  Toy / Game Store  Trail  \
0               0.0                     0.0               0.0    0.0   
1               0.0                     0.0               0.0    0.0   
2               0.0                     0.0               0.0    0.0   
3               0.0                     0.0               0.0    0.0   
4               0.0                     0.0               0.0    0.0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0            0.0                       0.043478               0.0   
1            0.0                       0.000000               0.0   
2            0.0                       0.000000               0.0   
3            0.0                       0.015873               0.0   
4            0.0                       0.000000               0.0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  
0                    0.0  0.000000        0.0  
1                    0.0  0.000000        0.0  
2                    0.0  0.000000        0.0  
3                    0.0  0.015873        0.0  
4                    0.0  0.000000        0.0  

[5 rows x 220 columns]

### Let's print each neighborhood along with the top 5 most common venues.

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.09
2          Bakery  0.07
3  Sandwich Place  0.07
4        Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0       Breakfast Spot  0.10
1       Sandwich Place  0.10
2                 Café  0.10
3          Coffee Shop  0.10
4  Japanese Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.08
2    Sushi Restaurant  0.06
3                Café  0.05
4  Italian Restaurant  0.05


----Christie----
                venue  freq
0       Grocery Store  0.27
1                Café  0.20
2                Park  0.13
3  Ath

### Putting that into a pandas dataframe.

In [40]:
# sorts the venus in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [41]:
import numpy as np


In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place        Breakfast Spot   
2  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie         Grocery Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar                Bakery        Sandwich Place   
1        Sandwich Place                  Café           Coffee Shop   
2        Airport Lounge      Airport Terminal       Harbor / Marina   
3        Sandwich Place      Sushi Restaurant                  Café   
4                  Café                  Park    Athletics & Sports   

  5th Most Common Venue          6th Most Common Venue 7th Most Common Venue  \
0              Beer Bar  Vegetarian / Vegan Restaurant        Farmers Market   
1   Japanese Restaurant                     Restaurant               Stadium   
2                   Bar                    Coffee Shop   Rental Car Location   
3    Italian Restaurant            Japanese Restaurant          Burger Joint   
4            Baby Store             Italian Restaurant             Nightclub   

    8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0      Seafood Restaurant                   Spa                    Butcher  
1  Furniture / Home Store                   Bar                     Bakery  
2        Sculpture Garden              Boutique              Boat or Ferry  
3             Salad Place            Restaurant                Pizza Place  
4              Restaurant           Coffee Shop  Middle Eastern Restaurant

## Clustering Neighborhoods. Will run k-means to cluster the neighborhood into 5 clusters. For that, it is necessary to import kmeans from scikit-learn

In [43]:
from sklearn.cluster import KMeans


In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Now, let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()


PostalCode                 Borough                    Neighborhood  \
0        M4E            East Toronto                     The Beaches   
1        M4J  East York/East Toronto              The Danforth  East   
2        M4K            East Toronto    The Danforth West, Riverdale   
3        M4L            East Toronto  India Bazaar, The Beaches West   
4        M4M            East Toronto                 Studio District   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.676357 -79.293031             0.0     Health Food Store   
1  43.685347 -79.338106             0.0                  Park   
2  43.679557 -79.352188             0.0      Greek Restaurant   
3  43.668999 -79.315572             0.0  Fast Food Restaurant   
4  43.659526 -79.340923             0.0           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Pub         Movie Theater   Martial Arts School   
1     Convenience Store         Metro Station           Yoga Studio   
2    Italian Restaurant           Coffee Shop        Ice Cream Shop   
3                  Park            Board Shop    Italian Restaurant   
4             Gastropub                Bakery                  Café   

      5th Most Common Venue   6th Most Common Venue     7th Most Common Venue  \
0  Mediterranean Restaurant             Men's Store             Metro Station   
1       Moroccan Restaurant     Martial Arts School  Mediterranean Restaurant   
2               Yoga Studio  Furniture / Home Store                      Bank   
3                       Pub           Movie Theater                Restaurant   
4        Seafood Restaurant  Thrift / Vintage Store                       Bar   

  8th Most Common Venue      9th Most Common Venue     10th Most Common Venue  
0    Mexican Restaurant  Middle Eastern Restaurant         Miscellaneous Shop  
1           Men's Store         Mexican Restaurant  Middle Eastern Restaurant  
2        Sandwich Place                        Pub                        Spa  
3        Sandwich Place              Burrito Place                    Brewery  
4                  Bank                Cheese Shop           Stationery Store


### Finally, let's visualize the resulting clusters. In order to do that, matplotlib will be required. The math module will help identifying if there is any "NaN" along with the data.

In [47]:
import math
import matplotlib.cm as cm
import matplotlib.colors as colors


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], 
                                  toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], 
                                  toronto_merged['Cluster Labels']):
    if math.isnan(float(cluster)) is False:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    else:
        pass

map_clusters


## Examining Clusters. Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, it is possible to assign a name to each cluster.

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0             East Toronto             0.0     Health Food Store   
1   East York/East Toronto             0.0                  Park   
2             East Toronto             0.0      Greek Restaurant   
3             East Toronto             0.0  Fast Food Restaurant   
4             East Toronto             0.0           Coffee Shop   
6          Central Toronto             0.0     Food & Drink Shop   
7          Central Toronto             0.0        Clothing Store   
8          Central Toronto             0.0           Pizza Place   
10         Central Toronto             0.0           Coffee Shop   
12        Downtown Toronto             0.0           Coffee Shop   
13        Downtown Toronto             0.0      Sushi Restaurant   
14        Downtown Toronto             0.0           Coffee Shop   
15        Downtown Toronto             0.0           Coffee Shop   
16        Downtown Toronto             0.0           Coffee Shop   
17        Downtown Toronto             0.0           Coffee Shop   
18        Downtown Toronto             0.0           Coffee Shop   
19        Downtown Toronto             0.0           Coffee Shop   
20        Downtown Toronto             0.0           Coffee Shop   
21        Downtown Toronto             0.0           Coffee Shop   
22        Downtown Toronto             0.0           Coffee Shop   
25         Central Toronto             0.0        Sandwich Place   
26        Downtown Toronto             0.0                  Café   
27        Downtown Toronto             0.0                  Café   
28        Downtown Toronto             0.0       Airport Service   
29  Downtown Toronto Stn A             0.0        Sandwich Place   
30        Downtown Toronto             0.0           Coffee Shop   
31        Downtown Toronto             0.0         Grocery Store   
32            West Toronto             0.0                Bakery   
33            West Toronto             0.0                   Bar   
34            West Toronto             0.0        Breakfast Spot   
35            West Toronto             0.0                   Bar   
36            West Toronto             0.0           Coffee Shop   
37            West Toronto             0.0           Coffee Shop   
38   East Toronto Business             0.0           Yoga Studio   

            2nd Most Common Venue          3rd Most Common Venue  \
0                             Pub                  Movie Theater   
1               Convenience Store                  Metro Station   
2              Italian Restaurant                    Coffee Shop   
3                            Park                     Board Shop   
4                       Gastropub                         Bakery   
6                    Dance Studio                           Park   
7                     Coffee Shop                            Spa   
8                    Dessert Shop                    Coffee Shop   
10            American Restaurant                           Bank   
12                         Bakery             Italian Restaurant   
13            Japanese Restaurant                        Gay Bar   
14                           Park                            Pub   
15                 Sandwich Place                 Clothing Store   
16                           Café             Italian Restaurant   
17                   Cocktail Bar                         Bakery   
18                 Sandwich Place               Sushi Restaurant   
19                           Café                 Sandwich Place   
20                          Hotel                           Café   
21                           Café                          Hotel   
22                           Café                 Sandwich Place   
25                           Park                    Coffee Shop   
26                 Sandwich Place                    Coffee Shop   
27  Vegetarian / Vegan Restaurant                    Coffee Shop   
28    

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
23  Central Toronto             1.0                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23          Home Service  Fast Food Restaurant           Yoga Studio   

   5th Most Common Venue     6th Most Common Venue 7th Most Common Venue  \
23   Martial Arts School  Mediterranean Restaurant           Men's Store   

   8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
23         Metro Station    Mexican Restaurant  Middle Eastern Restaurant

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
24  Central Toronto             2.0                 Trail   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
24         Jewelry Store              Bus Line      Sushi Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
24           Yoga Studio   Monument / Landmark   Martial Arts School   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
24  Mediterranean Restaurant           Men's Store          Metro Station

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
5  Central Toronto             3.0    Dim Sum Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
5                  Park              Bus Line           Swim School   

  5th Most Common Venue     6th Most Common Venue 7th Most Common Venue  \
5   Moroccan Restaurant  Mediterranean Restaurant           Men's Store   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
5         Metro Station    Mexican Restaurant  Middle Eastern Restaurant

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
11  Downtown Toronto             4.0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
11            Playground                 Trail           Yoga Studio   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
11   Monument / Landmark      Malay Restaurant   Martial Arts School   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
11  Mediterranean Restaurant           Men's Store          Metro Station